# Video Autoencoder


In [ ]:
%load_ext autoreload
%autoreload 2
from persim import plot_diagrams
from takenslayers import *
from takenspipelines import *
from videoutils import *
import matplotlib.pyplot as plt

import math

import numpy as np
from scipy.signal import convolve
from skimage.transform import resize

import torch
from torch.autograd import Variable
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torchvision.transforms as T

torch.manual_seed(2)

In [ ]:
frames = load_video("Videos/face.mp4", make_rgb=True)
X = np.moveaxis(frames, [0, 1, 2, 3], [2, 3, 1, 0])
X = np.array(X, dtype=float)/255
X = X[:, :, 0:512, 0:512]
plt.imshow(np.moveaxis(X[0, :, :, :], (0, 1, 2), (2, 0, 1)))
print("X.shape", X.shape)
data = torch.from_numpy(X).float()

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return 'cpu'#device
device = get_device()
print(device)
data = data.to(device)
autoencoder = VideoAutoencoderCNN(device, data, depth=3, dim=100, lr=0.001, last_layer=nn.Sigmoid)
autoencoder = autoencoder.to(device)
y = autoencoder.train_epochs(300, plot_interval=1)

In [ ]:
res = np.array(y.cpu().detach().numpy())
frames_res = np.moveaxis(res, (2, 3, 0, 1), (0, 1, 3, 2))
frames_res = np.array(frames_res*255, dtype=np.uint8)
frames_res[frames_res < 0] = 0
frames_res[frames_res > 255] = 255
save_video("result.avi", frames_res, is_rgb=True)